<a href="https://colab.research.google.com/github/daisyKim12/Tensorflow_Study/blob/main/Image_Classification_German_Traffic_Signs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# downgrade google colab tensorflow version to 2.9.0
import urllib.request

url = 'https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/tfcert.py'
urllib.request.urlretrieve(url, 'tfcert.py')
%run tfcert.py

import tensorflow as tf
import tensorflow_datasets as tfds

설치환경: Google Colab
TensorFlow 시험환경을 구성중입니다. 잠시만 기다려 주세요.
(설치는 약 1~5분 정도 소요 됩니다)
[알림] TensorFlow 시험환경 구성이 완료 되었습니다.


In [4]:
import urllib
import zipfile

import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

In [5]:
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/germantrafficsigns.zip'
    urllib.request.urlretrieve(url, 'germantrafficsigns.zip')
    with zipfile.ZipFile('germantrafficsigns.zip', 'r') as zip_ref:
        zip_ref.extractall()

In [6]:
def preprocess(image, label):
    # NORMALIZE
    image /= 255

    return image, label

In [7]:
download_and_extract_data()
BATCH_SIZE = 32
IMG_SIZE = 30

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='train/',
    label_mode='categorical',
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size = BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='validation/',
    label_mode='categorical',
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size = BATCH_SIZE)

Found 31367 files belonging to 43 classes.
Found 7842 files belonging to 43 classes.


In [10]:
train_ds = train_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
        tf.data.experimental.AUTOTUNE)
val_ds = val_ds.map(
    preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [13]:
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

In [14]:
model = tf.keras.models.Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(30,30,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.25),
    Dense(128, activation='relu'),
    Dense(43, activation='softmax'),
])

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [17]:
checkpoint_path = 'tmp_checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only = True,
                             save_weights_only = True,
                             monitor='val_loss',
                             verbose=1)

In [18]:
model.fit(train_ds, validation_data = val_ds,
          epochs = 5, callbacks = [checkpoint])

Epoch 1/5
981/981 [==============================] - ETA: 0s - loss: 1.8052 - acc: 0.4847
Epoch 1: val_loss improved from inf to 0.64473, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 24s 15ms/step - loss: 1.8052 - acc: 0.4847 - val_loss: 0.6447 - val_acc: 0.8154
Epoch 2/5
979/981 [============================>.] - ETA: 0s - loss: 0.5476 - acc: 0.8305
Epoch 2: val_loss improved from 0.64473 to 0.26336, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 9s 9ms/step - loss: 0.5473 - acc: 0.8306 - val_loss: 0.2634 - val_acc: 0.9218
Epoch 3/5
976/981 [============================>.] - ETA: 0s - loss: 0.3156 - acc: 0.9053
Epoch 3: val_loss improved from 0.26336 to 0.16989, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 12s 12ms/step - loss: 0.3152 - acc: 0.9054 - val_loss: 0.1699 - val_acc: 0.9515
Epoch 4/5
980/981 [============================>.] - ETA: 0s - loss: 0.2174 - acc: 0.9335
Epoch 4: val_

In [19]:
model.load_weights(checkpoint_path)

In [20]:
model.evaluate(val_ds)

246/246 [==============================] - 2s 10ms/step - loss: 0.0995 - acc: 0.9717


[0.09952019900083542, 0.9716908931732178]